# Robustness Analysis and Benchmarking

This notebook covers:
- Filter function analysis
- Randomized benchmarking
- Sensitivity and worst-case analysis
- Noise spectroscopy

In [ ]:
import sys
sys.path.insert(0, '..')
import numpy as np
import matplotlib.pyplot as plt
import qutip as qt

from src.optimization.filter_functions import (
    compute_filter_function,
    PowerSpectralDensity,
    compute_infidelity,
    visualize_filter_function
)
from src.optimization.benchmarking import (
    RBExperiment,
    generate_rb_sequence,
    run_interleaved_rb
)
from src.optimization.robustness import (
    compute_fidelity_robustness,
    sensitivity_analysis,
    compute_fisher_information
)

%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 8)

## 1. Filter Function Analysis

Filter functions quantify pulse sensitivity to noise at different frequencies.

In [ ]:
# Create example pulse
times = np.linspace(0, 10, 100)
pulse = np.exp(-(times-5)**2/2) * np.sin(2*np.pi*times/10)

# Compute filter function
frequencies, filter_func = compute_filter_function(pulse, times)

# Visualize
fig = visualize_filter_function(frequencies, filter_func, pulse, times)
plt.show()

print(f"Peak filter function: {np.max(filter_func):.4f}")
print(f"Dominant frequency: {frequencies[np.argmax(filter_func)]:.4f} GHz")

## 2. Randomized Benchmarking

Standard protocol for characterizing average gate fidelity.

In [ ]:
# Setup RB experiment
sequence_lengths = [1, 2, 5, 10, 20, 40, 80]
n_sequences = 20

rb_exp = RBExperiment(
    n_qubits=1,
    sequence_lengths=sequence_lengths,
    n_sequences_per_length=n_sequences
)

# Generate and run sequences
print("Running randomized benchmarking...")
results = rb_exp.run_experiment()

# Fit decay
p_fit, F_avg = rb_exp.fit_decay(results)

# Plot
fig = rb_exp.plot_results(results, p_fit, F_avg)
plt.show()

print(f"\nRB Results:")
print(f"  Decay parameter p: {p_fit:.6f}")
print(f"  Average gate fidelity: {F_avg:.6f}")
print(f"  Error per gate: {1-F_avg:.6f}")

## 3. Sensitivity Analysis

In [ ]:
# Analyze sensitivity to system parameters
params = {'amplitude': 1.0, 'detuning': 0.0, 'duration': 10.0}
param_variations = {'amplitude': 0.1, 'detuning': 0.05}

sensitivities = sensitivity_analysis(pulse, times, params, param_variations)

print("\nParameter Sensitivities:")
for param, sens in sensitivities.items():
    print(f"  {param}: {sens:.6f}")

## Summary

- **Filter functions** identify noise-sensitive frequencies
- **RB** provides model-independent fidelity estimates
- **Sensitivity analysis** guides robustness optimization